In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# number of samples to calculate validation and accuracy
# decrease this if you're running out of memory
test_valid_size = 256

# network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # dropout (probability to keep units)

/Users/KaoruOta/opt/anaconda3/envs/deep_learning_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# store weights & biases
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [4]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [5]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch  1, Batch   1 -Loss: 113141.4922 Validation Accuracy: 0.050781
Epoch  1, Batch   2 -Loss: 72911.2969 Validation Accuracy: 0.035156
Epoch  1, Batch   3 -Loss: 58442.4961 Validation Accuracy: 0.035156
Epoch  1, Batch   4 -Loss: 51514.8867 Validation Accuracy: 0.082031
Epoch  1, Batch   5 -Loss: 42763.0859 Validation Accuracy: 0.085938
Epoch  1, Batch   6 -Loss: 39148.3203 Validation Accuracy: 0.078125
Epoch  1, Batch   7 -Loss: 39177.2422 Validation Accuracy: 0.082031
Epoch  1, Batch   8 -Loss: 32398.0000 Validation Accuracy: 0.101562
Epoch  1, Batch   9 -Loss: 31088.6133 Validation Accuracy: 0.097656
Epoch  1, Batch  10 -Loss: 29747.0137 Validation Accuracy: 0.097656
Epoch  1, Batch  11 -Loss: 24504.3633 Validation Accuracy: 0.121094
Epoch  1, Batch  12 -Loss: 24002.6895 Validation Accuracy: 

Epoch  1, Batch 116 -Loss:  4785.6841 Validation Accuracy: 0.535156
Epoch  1, Batch 117 -Loss:  4815.3364 Validation Accuracy: 0.531250
Epoch  1, Batch 118 -Loss:  3412.6372 Validation Accuracy: 0.542969
Epoch  1, Batch 119 -Loss:  4864.4551 Validation Accuracy: 0.550781
Epoch  1, Batch 120 -Loss:  4585.4146 Validation Accuracy: 0.554688
Epoch  1, Batch 121 -Loss:  4091.8145 Validation Accuracy: 0.566406
Epoch  1, Batch 122 -Loss:  5279.9854 Validation Accuracy: 0.570312
Epoch  1, Batch 123 -Loss:  3937.7646 Validation Accuracy: 0.570312
Epoch  1, Batch 124 -Loss:  3854.2031 Validation Accuracy: 0.570312
Epoch  1, Batch 125 -Loss:  4565.7520 Validation Accuracy: 0.574219
Epoch  1, Batch 126 -Loss:  3739.1050 Validation Accuracy: 0.570312
Epoch  1, Batch 127 -Loss:  4769.3652 Validation Accuracy: 0.585938
Epoch  1, Batch 128 -Loss:  3720.5928 Validation Accuracy: 0.578125
Epoch  1, Batch 129 -Loss:  5286.6138 Validation Accuracy: 0.578125
Epoch  1, Batch 130 -Loss:  4039.5923 Validation

Epoch  1, Batch 237 -Loss:  1878.9216 Validation Accuracy: 0.652344
Epoch  1, Batch 238 -Loss:  3034.2251 Validation Accuracy: 0.652344
Epoch  1, Batch 239 -Loss:  1889.0579 Validation Accuracy: 0.648438
Epoch  1, Batch 240 -Loss:  2972.9136 Validation Accuracy: 0.652344
Epoch  1, Batch 241 -Loss:  2202.0562 Validation Accuracy: 0.648438
Epoch  1, Batch 242 -Loss:  2557.8582 Validation Accuracy: 0.660156
Epoch  1, Batch 243 -Loss:  2521.9375 Validation Accuracy: 0.660156
Epoch  1, Batch 244 -Loss:  2616.2292 Validation Accuracy: 0.664062
Epoch  1, Batch 245 -Loss:  2349.6191 Validation Accuracy: 0.648438
Epoch  1, Batch 246 -Loss:  2318.0957 Validation Accuracy: 0.656250
Epoch  1, Batch 247 -Loss:  2526.4072 Validation Accuracy: 0.667969
Epoch  1, Batch 248 -Loss:  2287.9775 Validation Accuracy: 0.656250
Epoch  1, Batch 249 -Loss:  2308.3047 Validation Accuracy: 0.660156
Epoch  1, Batch 250 -Loss:  2733.5947 Validation Accuracy: 0.652344
Epoch  1, Batch 251 -Loss:  1868.8113 Validation

Epoch  1, Batch 358 -Loss:  1692.1725 Validation Accuracy: 0.699219
Epoch  1, Batch 359 -Loss:  1620.3176 Validation Accuracy: 0.695312
Epoch  1, Batch 360 -Loss:  1816.2319 Validation Accuracy: 0.703125
Epoch  1, Batch 361 -Loss:  1495.2144 Validation Accuracy: 0.707031
Epoch  1, Batch 362 -Loss:  2072.1133 Validation Accuracy: 0.703125
Epoch  1, Batch 363 -Loss:  1354.3760 Validation Accuracy: 0.703125
Epoch  1, Batch 364 -Loss:  1405.1566 Validation Accuracy: 0.699219
Epoch  1, Batch 365 -Loss:  1845.6727 Validation Accuracy: 0.699219
Epoch  1, Batch 366 -Loss:  1773.6093 Validation Accuracy: 0.695312
Epoch  1, Batch 367 -Loss:  1719.0209 Validation Accuracy: 0.695312
Epoch  1, Batch 368 -Loss:  1890.4237 Validation Accuracy: 0.695312
Epoch  1, Batch 369 -Loss:  1617.1807 Validation Accuracy: 0.707031
Epoch  1, Batch 370 -Loss:  1347.4937 Validation Accuracy: 0.710938
Epoch  1, Batch 371 -Loss:  1898.2079 Validation Accuracy: 0.707031
Epoch  1, Batch 372 -Loss:  2278.6406 Validation

Epoch  2, Batch  50 -Loss:  1358.3801 Validation Accuracy: 0.734375
Epoch  2, Batch  51 -Loss:  1162.1273 Validation Accuracy: 0.734375
Epoch  2, Batch  52 -Loss:  1076.6567 Validation Accuracy: 0.734375
Epoch  2, Batch  53 -Loss:  1557.5168 Validation Accuracy: 0.742188
Epoch  2, Batch  54 -Loss:  1459.8008 Validation Accuracy: 0.742188
Epoch  2, Batch  55 -Loss:  1361.4124 Validation Accuracy: 0.738281
Epoch  2, Batch  56 -Loss:  1601.9445 Validation Accuracy: 0.738281
Epoch  2, Batch  57 -Loss:  1484.2648 Validation Accuracy: 0.738281
Epoch  2, Batch  58 -Loss:  1366.1604 Validation Accuracy: 0.742188
Epoch  2, Batch  59 -Loss:  1407.6067 Validation Accuracy: 0.742188
Epoch  2, Batch  60 -Loss:  1379.2894 Validation Accuracy: 0.738281
Epoch  2, Batch  61 -Loss:  1681.6396 Validation Accuracy: 0.742188
Epoch  2, Batch  62 -Loss:  1390.2555 Validation Accuracy: 0.750000
Epoch  2, Batch  63 -Loss:   994.2135 Validation Accuracy: 0.742188
Epoch  2, Batch  64 -Loss:  1289.0867 Validation

...

Epoch  8, Batch 138 -Loss:   292.8738 Validation Accuracy: 0.804688
Epoch  8, Batch 139 -Loss:   334.5243 Validation Accuracy: 0.812500
Epoch  8, Batch 140 -Loss:   200.4296 Validation Accuracy: 0.804688
Epoch  8, Batch 141 -Loss:   259.5965 Validation Accuracy: 0.800781
Epoch  8, Batch 142 -Loss:   293.0881 Validation Accuracy: 0.800781
Epoch  8, Batch 143 -Loss:   312.5609 Validation Accuracy: 0.796875
Epoch  8, Batch 144 -Loss:   289.6155 Validation Accuracy: 0.796875
Epoch  8, Batch 145 -Loss:   221.3362 Validation Accuracy: 0.796875
Epoch  8, Batch 146 -Loss:   345.5806 Validation Accuracy: 0.796875
Epoch  8, Batch 147 -Loss:   203.8133 Validation Accuracy: 0.800781
Epoch  8, Batch 148 -Loss:   364.1539 Validation Accuracy: 0.804688
Epoch  8, Batch 149 -Loss:   302.0675 Validation Accuracy: 0.808594
Epoch  8, Batch 150 -Loss:   297.8370 Validation Accuracy: 0.808594
Epoch  8, Batch 151 -Loss:   364.6891 Validation Accuracy: 0.796875
Epoch  8, Batch 152 -Loss:   420.4330 Validation

Epoch  8, Batch 259 -Loss:   187.3741 Validation Accuracy: 0.804688
Epoch  8, Batch 260 -Loss:   352.5170 Validation Accuracy: 0.800781
Epoch  8, Batch 261 -Loss:   333.6002 Validation Accuracy: 0.804688
Epoch  8, Batch 262 -Loss:   297.1068 Validation Accuracy: 0.800781
Epoch  8, Batch 263 -Loss:   216.3252 Validation Accuracy: 0.800781
Epoch  8, Batch 264 -Loss:   232.4724 Validation Accuracy: 0.800781
Epoch  8, Batch 265 -Loss:   202.8326 Validation Accuracy: 0.800781
Epoch  8, Batch 266 -Loss:   315.8159 Validation Accuracy: 0.796875
Epoch  8, Batch 267 -Loss:   254.4702 Validation Accuracy: 0.796875
Epoch  8, Batch 268 -Loss:   218.7133 Validation Accuracy: 0.796875
Epoch  8, Batch 269 -Loss:   325.4025 Validation Accuracy: 0.800781
Epoch  8, Batch 270 -Loss:   380.2601 Validation Accuracy: 0.800781
Epoch  8, Batch 271 -Loss:   340.3709 Validation Accuracy: 0.800781
Epoch  8, Batch 272 -Loss:   362.1578 Validation Accuracy: 0.800781
Epoch  8, Batch 273 -Loss:   250.4545 Validation

Epoch  8, Batch 380 -Loss:   381.2209 Validation Accuracy: 0.804688
Epoch  8, Batch 381 -Loss:   177.5096 Validation Accuracy: 0.800781
Epoch  8, Batch 382 -Loss:   172.2422 Validation Accuracy: 0.800781
Epoch  8, Batch 383 -Loss:   169.9983 Validation Accuracy: 0.800781
Epoch  8, Batch 384 -Loss:   153.0331 Validation Accuracy: 0.804688
Epoch  8, Batch 385 -Loss:   341.7151 Validation Accuracy: 0.800781
Epoch  8, Batch 386 -Loss:   257.0303 Validation Accuracy: 0.804688
Epoch  8, Batch 387 -Loss:   318.1899 Validation Accuracy: 0.800781
Epoch  8, Batch 388 -Loss:   216.8702 Validation Accuracy: 0.800781
Epoch  8, Batch 389 -Loss:   164.5487 Validation Accuracy: 0.804688
Epoch  8, Batch 390 -Loss:   316.7807 Validation Accuracy: 0.808594
Epoch  8, Batch 391 -Loss:   294.4233 Validation Accuracy: 0.808594
Epoch  8, Batch 392 -Loss:   243.7739 Validation Accuracy: 0.812500
Epoch  8, Batch 393 -Loss:   223.0402 Validation Accuracy: 0.812500
Epoch  8, Batch 394 -Loss:   301.5220 Validation

Epoch  9, Batch  72 -Loss:   298.6672 Validation Accuracy: 0.796875
Epoch  9, Batch  73 -Loss:   222.1031 Validation Accuracy: 0.796875
Epoch  9, Batch  74 -Loss:   371.1518 Validation Accuracy: 0.796875
Epoch  9, Batch  75 -Loss:   366.2900 Validation Accuracy: 0.792969
Epoch  9, Batch  76 -Loss:   233.5298 Validation Accuracy: 0.796875
Epoch  9, Batch  77 -Loss:   354.5021 Validation Accuracy: 0.796875
Epoch  9, Batch  78 -Loss:   264.4749 Validation Accuracy: 0.796875
Epoch  9, Batch  79 -Loss:   308.8353 Validation Accuracy: 0.800781
Epoch  9, Batch  80 -Loss:   259.7527 Validation Accuracy: 0.792969
Epoch  9, Batch  81 -Loss:   302.0921 Validation Accuracy: 0.792969
Epoch  9, Batch  82 -Loss:   236.6388 Validation Accuracy: 0.796875


KeyboardInterrupt: 